In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

from mpmath import besseljzero
from scipy.special import gamma
from scipy.special import jv, iv, ive
from scipy.optimize import differential_evolution
from scipy.interpolate import interp1d

from scipy.stats import pearsonr
from sklearn.metrics import r2_score

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
def simulate_HSDM_3D(a, mu, eta, ndt, sigma=1, dt=0.001):
    x = np.zeros(mu.shape)
    
    rt = 0
    
    mu_t = np.random.normal(mu, eta) 
    while np.linalg.norm(x, 2) < a(rt):
        x += mu_t*dt + sigma*np.sqrt(dt)*np.random.normal(0, 1, mu.shape)
        rt += dt
    
    theta1 = np.arctan2(np.sqrt(x[2]**2 + x[1]**2), x[0])
    theta2 = np.arctan2(x[2], x[1])   
    
    return rt+ndt, (theta1, theta2)

In [3]:
def k(a, da, t, q, sigma=2):
    return 0.5 * (q - 0.5*sigma - da(t))

def psi(a, da, t, z, tau, q, sigma=2):
    kk = k(a, da, t, q, sigma)
    
    if 2*np.sqrt(a(t)*z)/(sigma*(t-tau))<=700:
        term1 = 1./(sigma*(t - tau)) * np.exp(- (a(t) + z)/(sigma*(t-tau)))
        term2 = (a(t)/z)**(0.5*(q-sigma)/sigma)
        term3 = da(t) - (a(t)/(t-tau)) + kk
        term4 = iv(q/sigma-1, 2*np.sqrt(a(t)*z)/(sigma*(t-tau)))
        term5 = (np.sqrt(a(t)*z)/(t-tau)) * iv(q/sigma, 2*np.sqrt(a(t)*z)/(sigma*(t-tau)))
    else:
        term1 = 1./(sigma*(t - tau))
        term2 = (a(t)/z)**(0.5*(q-sigma)/sigma)
        term3 = da(t) - (a(t)/(t-tau)) + kk
        term4 = ive(q/sigma-1, (a(t) + z)/(sigma*(t-tau)))
        term5 = (np.sqrt(a(t)*z)/(t-tau)) * ive(q/sigma, (a(t) + z)/(sigma*(t-tau)))
    
    return term1 * term2 * (term3 * term4 + term5)

def ie_bessel_fpt(a, da, q, z, sigma=2, dt=0.1, T_max=2):
    g = [0]
    T = [0]
    g.append(-2*psi(a, da, dt, z, 0, q, sigma))
    T.append(dt)
    
    for n in range(2, int(T_max/dt)+2):
        s = -2 * psi(a, da, n*dt, z, 0, q, sigma)

        for j in range(1, n):
            s += 2 * dt * g[j] * psi(a, da, n*dt, a(j*dt), j*dt, q, sigma)

        g.append(s)
        T.append(n*dt)
        
    g = np.asarray(g)
    T = np.asarray(T)
    
    gt = interp1d(T, g)
    return gt

In [4]:
def HSDM_3D_likelihood(prms, RT, Theta):
    a = lambda t: prms[0]**2
    da = lambda t: 0
    ndt = prms[1]
    
    eta = prms[2]
    eta2 = eta**2
    
    mu = np.array([prms[3], prms[4], prms[5]])
    
    if max(RT) <= 4.5:
        fpt = ie_bessel_fpt(a, da, mu.shape[0], 0.000001, 
                            dt=0.02, T_max=max(RT))
    else:
        fpt = ie_bessel_fpt(a, da, mu.shape[0], 0.000001, 
                            dt=0.02, T_max=4.5)
        
        fpt_l = ie_bessel_fpt(a, da, mu.shape[0], 0.000001, 
                              dt=0.1, T_max=max(RT))
    log_lik = 0
    for i in range(len(RT)):
        rt, theta = RT[i], Theta[i]
        if rt - ndt > 0.001 and ndt >= 0:
            x2 =  prms[0]*np.cos(theta[0])
            x1 =  prms[0]*np.sin(theta[0])*np.cos(theta[1]) 
            x0 =  prms[0]*np.sin(theta[0])*np.sin(theta[1])
            fixed = 1/(np.sqrt(eta2 * (rt - ndt) + 1))
            exponent0 = -0.5*mu[0]**2/eta2 + 0.5*(x0 * eta2 + mu[0])**2 / (eta2 * (eta2 * (rt - ndt) + 1))
            exponent1 = -0.5*mu[1]**2/eta2 + 0.5*(x1 * eta2 + mu[1])**2 / (eta2 * (eta2 * (rt - ndt) + 1))
            exponent2 = -0.5*mu[2]**2/eta2 + 0.5*(x2 * eta2 + mu[2])**2 / (eta2 * (eta2 * (rt - ndt) + 1))
            term1 = fixed * np.exp(exponent0)
            term2 = fixed * np.exp(exponent1)
            term3 = fixed * np.exp(exponent2)
            
            if rt - ndt <= 4.5:
                density = term1 * term2 * term3 * fpt(rt - ndt)
            else:
                density = term1 * term2 * term3 * fpt_l(rt - ndt)
            
            if 0.1**14 < density:
                log_lik += -np.log(density)
            else:
                log_lik += -np.log(0.1**14)
        else:
            log_lik += -np.log(0.1**14)
        
    return log_lik

In [5]:
min_threshold = 1
max_threshold = 3

min_ndt = 0.1
max_ndt = 1

min_eta = 0.1
max_eta = 1

min_mu = -2.5
max_mu = 2.5

file_name = '_Recovery_data/IE_3d_recovery_dvar.csv'

In [6]:
for n in tqdm(range(50)):
    threshold = np.random.uniform(min_threshold, max_threshold)
    a = lambda t: threshold
    ndt = np.random.uniform(min_ndt, max_ndt)
    eta = np.random.uniform(min_eta, max_eta)
    mu = np.array([np.random.uniform(min_mu, max_mu), 
                   np.random.uniform(min_mu, max_mu),
                   np.random.uniform(min_mu, max_mu)])    
    
    RT = []
    Theta = []
    
    for i in range(500):
        rt, theta = simulate_HSDM_3D(a, mu, eta, ndt)
        RT.append(rt)
        Theta.append(theta)
    
    min_ans = differential_evolution(HSDM_3D_likelihood,
                                     args=(RT, Theta), 
                                     bounds=[(min_threshold, max_threshold), 
                                             (min_ndt, max_ndt), (min_eta, max_eta),
                                             (min_mu, max_mu), (min_mu, max_mu), (min_mu, max_mu)])
    
    if min_ans.success:
        recovery_df = pd.DataFrame({'threshold_true': [threshold],
                                    'threshold_estimate': [min_ans.x[0]],
                                    'ndt_true': [ndt],
                                    'ndt_estimate': [min_ans.x[1]],
                                    'eta_true': [eta],
                                    'eta_estimate': [min_ans.x[2]],
                                    'mu1_true': [mu[2]],
                                    'mu1_estimate': [min_ans.x[3]],
                                    'mu2_true': [mu[1]],
                                    'mu2_estimate': [min_ans.x[4]],
                                    'mu3_true': [mu[0]],
                                    'mu3_estimate': [min_ans.x[5]]})
        
        print(eta, min_ans.x[2])
        
        old_recovery_data = pd.read_csv(file_name, index_col=0)
        recovery_df = pd.concat([old_recovery_data, recovery_df]).reset_index(drop=True)
        recovery_df.to_csv(file_name)

  2%|▌                              | 1/50 [05:09<4:12:23, 309.04s/it]

0.28320246421051487 0.34370600497979603


  4%|█▏                             | 2/50 [09:04<3:32:37, 265.79s/it]

0.939381154167334 1.0


  6%|█▊                             | 3/50 [11:09<2:37:59, 201.70s/it]

...

0.3871566351253087 0.6908476013747348


 76%|██████████████████████▊       | 38/50 [2:47:08<47:09, 235.79s/it]

0.13826496579313954 0.11403607080840449


 76%|██████████████████████▊       | 38/50 [2:49:46<53:36, 268.06s/it]


KeyboardInterrupt: 